In [1]:
from datasets import load_dataset

/home/bt19d200/NER_Vamshi/NER_Model/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("ade-benchmark-corpus/ade_corpus_v2",'Ade_corpus_v2_drug_ade_relation') 


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})

In [4]:
test = dataset["train"]
test[0]

{'text': 'Intravenous azithromycin-induced ototoxicity.',
 'drug': 'azithromycin',
 'effect': 'ototoxicity',
 'indexes': {'drug': {'start_char': [12], 'end_char': [24]},
  'effect': {'start_char': [33], 'end_char': [44]}}}

In [ ]:
from openai import OpenAI
import json
import time
from tqdm import tqdm
import pandas as pd  # make sure df is defined with 'Content' column

# Set OpenAI API key
client = OpenAI(api_key = "")  # Replace with your actual API key

# Function: extract named_entities given allowed entity labels
def extract_named_entities_by_labels(paragraph, allowed_labels):
    allowed_labels_str = ", ".join(allowed_labels)

    system_prompt = (
        "You are a named entity recognition (NER) assistant. "
        "Your task is to extract named entities from a given paragraph, "
        "but only include entities whose type is one of the following: "
        f"{allowed_labels_str}. "
        "Return ONLY a JSON list of entity strings that match the allowed labels. "
        "Do not include the labels in the output. Keep it JSON parsable."
    )

    user_prompt = f"""
Paragraph:

{paragraph}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        if reply.startswith("```json"):
            reply = reply[len("```json"):].strip()
        if reply.endswith("```"):
            reply = reply[:-len("```")].strip()

        return json.loads(reply)

    except Exception as e:
        print("Error with paragraph:", paragraph)
        print("Exception:", e)
        return []

    # Example: allowed labels
allowed_entity_labels = ['drug','effect']

# Loop through DataFrame and extract entities
gpt_named_entities = []

for i in tqdm(range(len(test))):
    paragraph = test[i]['text']
    entities = extract_named_entities_by_labels(paragraph, allowed_entity_labels)
    gpt_named_entities.append(entities)

100%|██████████| 6821/6821 [1:25:24<00:00,  1.33it/s]


In [12]:
with open("ade_gpt_named_entities_output.json", "w") as f:
    json.dump(gpt_named_entities, f, indent=2)

In [13]:
true=[]
for i in range(len(test)):
    temp=[test[i]['drug'],test[i]['effect']]
    true.append(temp)

In [14]:
true[6]

['naproxen', 'cutaneous fragility']

In [15]:
gpt_named_entities[6]

['naproxen', 'oxaprozin']

In [16]:
recall_avg=[]
precision_avg=[]
common2=0
for i in range(len(gpt_named_entities)):
    true_set=set(true[i])
    pred_set=set(gpt_named_entities[i])
    
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m in n or n in m:
                    count+= 1
        recall = count / len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(pred_set) > 0 else 0
        common2 += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [17]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.6980647998827152, 0.6144401583853251)

In [5]:
import re
ner=[]
for i in range(len(test)):
    temp=[]
    tokens = re.split(r'[ -.,:;/?\]\[]+', test[i]['text'])
    drug=re.split(r'[ -.,:;/?\]\[]+', test[i]['drug'])
    start=tokens.index(drug[0])
    end=tokens.index(drug[-1])
    temp.append([start,end,'drug',test[i]['drug']])
    effect=re.split(r'[ -.,:;/?\[\]]+', test[i]['effect'])
    start=tokens.index(effect[0])
    end=tokens.index(effect[-1])
    temp.append([start,end,'effect',test[i]['effect']])
    ner.append(temp)
test[0]

{'text': 'Intravenous azithromycin-induced ototoxicity.',
 'drug': 'azithromycin',
 'effect': 'ototoxicity',
 'indexes': {'drug': {'start_char': [12], 'end_char': [24]},
  'effect': {'start_char': [33], 'end_char': [44]}}}

In [18]:
ner[0]

[[1, 1, 'drug', 'azithromycin'], [3, 3, 'effect', 'ototoxicity']]

In [6]:
relations_true=[]
for i in range(len(ner)):
    temp=[]
    temp.append([ner[i][0][3], ner[i][1][3], 'has'])
    relations_true.append(temp)  

In [20]:
relations_true[3]

[['naproxen', 'pseudoporphyria', 'has']]

In [ ]:
from openai import OpenAI
import json
import re
from tqdm import tqdm

# Initialize OpenAI client
client = OpenAI(api_key = "")  # Replace with your actual API key

def extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels):
    """
    Extract RDF triples using GPT-4o/mini with entity spans.

    Args:
        paragraph (str): Full input paragraph
        entities (list): Each entity as [start, end, entity_type, entity_text]
        relation_labels (list): Allowed relation labels

    Returns:
        dict: {"relation_triples": [[head, relation, tail], ...]}
    """
    # Convert entity structure into readable text
    entity_descs = [
        f"[{start}, {end}, {etype}, {text}]"
        for start, end, etype, text in entities
    ]
    entity_str = "\n".join(entity_descs)
    relation_str = ", ".join(relation_labels)

    system_prompt = (
        "You are an expert in information extraction. "
        "Given a paragraph, a list of named entities with character spans and types, and a list of allowed relation labels, "
        "extract RDF relation triples in the format [head, relation, tail].\n\n"
        "- Head and tail must be from the provided entity list.\n"
        "- The relation must be from the relation_labels list.\n"
        "- The output must be ONLY a JSON object like:\n"
        '{ "relation_triples": [ ["Entity1", "Relation", "Entity2"], ... ] }\n'
        "- Do NOT include any explanation or extra text."
    )

    user_prompt = f"""
Paragraph:
\"\"\"
{paragraph}
\"\"\"

Entities (format: [start, end, type, entity]):
{entity_str}

Relation labels:
{relation_str}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        # Extract JSON block
        json_match = re.search(r'\{.*\}', reply, re.DOTALL)
        if json_match:
            json_text = json_match.group(0).strip()
            return json.loads(json_text)
        else:
            print("No JSON found.")
            print("Reply:", reply)
            return {"relation_triples": []}

    except Exception as e:
        print("Exception:", e)
        return {"relation_triples": []}
gpt_relation_triples = []
relation_labels = ['has']


for i in tqdm(range(len(test))):
    paragraph = test[i]['text']
    entities = ner[i]
    output= extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels)
    gpt_relation_triples.append(output['relation_triples'])

100%|██████████| 6821/6821 [1:51:24<00:00,  1.02it/s]  


In [9]:
with open("ade_gpt_relation_triples_output.json", "w") as f:
    json.dump(gpt_relation_triples, f, indent=2)

In [28]:
relations_true[6713]

[['calcitriol', 'secondary hyperparathyroidism', 'has']]

In [29]:
recall_avg=[]
precision_avg=[]
common=0
ours=0

for i in range(len(gpt_relation_triples)):
    true_set=relations_true[i]
    pred_set=gpt_relation_triples[i]
    d={}
    for item in pred_set:
        if (item[0],item[1]) not in d and (item[1],item[0]) not in d:
            d[(item[0],item[1])]=item[2]
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m[0]==n[0] and m[1]==n[2] and (m[2]==n[1]):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(d) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [30]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.9250842984899574, 0.924229096417925)

In [35]:
from pair2rel import Pair2Rel

from tqdm import tqdm
model = Pair2Rel.from_pretrained("chapalavamshi022/pair2rel")
import torch
import re
# Force usage of GPU 1
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.device = device 
relations_all=[]
labels = ['has']
for i in tqdm(range(len(test))):
    # required_labels = []
    # for token in processed_data[i]['tokens']:
    #     if token in rel_set:
    #         required_labels.append(token)
    tokens = re.split(r'[ -.,:;/?\]\[]+', test[i]['text'])
    try:

        relations = model.predict_relations(tokens, labels, threshold=0.0, ner=pred_ner[i], top_k=1)

        sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
        temp=[]
        for item in sorted_data_desc:
            head=' '.join(item['head_text'])
            tail=' '.join(item['tail_text'])
            if head == tail:
                continue
            temp.append([head,tail,item['label']])

        relations_all.append(temp)
    except:
        relations_all.append([])
        

print("Success! ✅")


/home/bt19d200/NER_Vamshi/NER_Model/.conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 6821/6821 [24:31<00:00,  4.64it/s]

Success! ✅


In [38]:
recall_avg=[]
precision_avg=[]
common=0
ours=0

for i in range(len(relations_all)):
    true_set=relations_true[i]
    pred_set=relations_all[i]
    d={}
    for item in pred_set:
        if (item[0],item[1]) not in d and (item[1],item[0]) not in d:
            d[(item[0],item[1])]=item[2]
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m[0] in n[0] and m[1] in n[1] and (m[2]==n[2]):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(d) if len(d) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [39]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.6116405219176074, 0.2600283059233326)